In [134]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic, needs_local_scope)
from ast import *
import numpy as np
from textwrap import indent

In [135]:
def print_message(test, message):
    if np.shape(message) != ():
        message = "\n".join(message)

    print("\u001b[35m\u001b[1m🐝 " + test)      
    print(indent(message, "      "))
    print("\u001b[0m")


In [204]:
@register_line_magic
@needs_local_scope
def check(line, local_ns=None):
    
    def test(op, left, right):
        t = type(op)
        if t == Lt:
            return left < right
        elif t == Eq:
            return left == right
        elif t == LtE:
            return left <= right
        else:
            return left != right

    def neg(op):
        t = type(op)
        if t == Lt:
            return ">="
        elif t == Eq:
            return "!="
        elif t == LtE:
            return ">"
        else:
            return "=="
        
    def my_eval(expr):
        body = expr.body
        message = [ ]
        if type(body) is Compare:
            left = body.left
            left_value = eval(compile(ast.unparse(left), '', 'eval'), None, local_ns)
            for op,right in zip(body.ops, body.comparators):
                right_value = eval(compile(ast.unparse(right), '', 'eval'), None, local_ns)
                x =  test(op, left_value, right_value)
                if not x:
                    m = ""
                    if type(left) != Constant:
                        m = m + f"{ast.unparse(left)} == {left_value} and "
                    if type(right) != Constant:
                        m = m + f"{ast.unparse(right)} == {right_value} and "                        
                    m = m + f'{left_value} {neg(op)} {right_value}'
                    message += [ m ]
                left, left_value = right, right_value
            if message != [ ]:
                print_message(f'check failed: {ast.unparse(expr)}', message)
        else:
            if not eval(ast.unparse(expr)):
                print_message(f'check failed: {ast.unparse(expr)}', 'Expression is not true')
    
    a = parse(line, mode='eval')
    my_eval(a)
del check

In [205]:
x = 2
y = 1
p = 0.5

def moo(x):
    return x % 2 == 0

In [211]:
from cs104 import *

check_less_than(0,x,2)
check_less_than(x,y)
check_equal(2,x)
check_greater_than(x,y)
check_between_or_equal(p, 0, 1)
check(True)
check(False)
check(moo(2))
check(moo(1))

🐝 check_less_than(0,x,2)
      x == 2 and 2 >= 2

🐝 check_less_than(x,y)
      x == 2 and y == 1 and 2 >= 1



NameError: name 'true' is not defined

In [209]:
%check 0 < x < 2
%check x < y
%check 2 == x > y
%check 0 <= p <= 1
%check True
%check False
%check moo(2)
%check moo(1)

🐝 check failed: 0 < x < 2
      x == 2 and 2 >= 2

🐝 check failed: x < y
      x == 2 and y == 1 and 2 >= 1

🐝 check failed: False
      Expression is not true

🐝 check failed: moo(1)
      Expression is not true

